# Example of building interface at the fractional coordinate z = 0.5

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import pickle
import random
import numpy as np

from random import sample
from collections import defaultdict

from pymatgen.core import Species, Composition
from pymatgen.core.structure import Structure

In [5]:
round_number = 4

nn_dist = 1.81865

nn1_dist = 2.97
nn2_dist = 4.2
nn3_dist = 5.14
nn4_dist = 5.94
    
Li = Species.from_str("Li+")
Mn3 = Species.from_str("Mn3+")
Mn4 = Species.from_str("Mn4+")
Ti4 = Species.from_str("Ti4+")
O2 = Species.from_str("O2-")
    
def Resources():          # Read resource file written by resource maker.

    with open('Resources.pickle', 'rb') as handle:
        Resources = pickle.load(handle)

    s = Resources['initial_structure']
    indices = Resources['indices'] 
    Occupied = Resources['Spinel_Orientation_Occupancies']
    
    for o in range(4):          # Deriving translational Variants of the 4 Rotational variants
        Occupied[o+4]= [x for x in indices['oct'] if x not in Occupied[o]]    
    
    #Nearest Neighbors lists and Neighbor Octahedra lists for octahedra are not are not extracted. But are available in the resources file.
    
    return Occupied, indices, s   

def Structure_Editor(s, indices):  #Editing structure so that sites on the edge and face of the cell are relocated to a consistent periodic boundary image.
    
    counter = 0

    for site in indices['oct']:      
        s[site].frac_coords = np.round(s[site].frac_coords,5)      
        Frac_Coords = s[site].frac_coords
        if Frac_Coords[0]==1 or Frac_Coords[1]==1 or Frac_Coords[2]==1:
            if Frac_Coords[0]==1:
                s[site].frac_coords[0]=0
            if Frac_Coords[1]==1:
                s[site].frac_coords[1]=0
            if Frac_Coords[2]==1:
                s[site].frac_coords[2]=0
                
    return s
    
def Interface_Cell_Occupancies(ordering1, ordering2, Occupied,central_flip, structure):
    
    counter1, counter2 = 0,0
    
    Mn_list, Vac_list = [],[]
    
    for octa in indices['oct']:
        if np.round(structure[octa].frac_coords[2],round_number)<central_flip:
            counter1+=1
            if (octa in Occupied[ordering1]):
                Mn_list.append(octa)
            else:
                Vac_list.append(octa)
        else:
            counter2+=1
            if (octa in Occupied[ordering2]):
                Mn_list.append(octa)
            else:
                Vac_list.append(octa)
                
    return  Mn_list, Vac_list

def Structure_Maker(Li_l, Vac_l, Mn_l,interfacial_poscar_filename,chgnet_structure):
    for li in Li_l:
        chgnet_structure.replace(li, {Li : 1}) 

    for mn3 in Mn_l:
        chgnet_structure.replace(mn3, {Mn3 : 1})

    chgnet_structure.remove_sites(Vac_l)  
    
    chgnet_structure = chgnet_structure.get_sorted_structure()
    
    chgnet_structure.to(fmt='poscar', filename=interfacial_poscar_filename)
    
def POSCAR_edit(poscar_file, a, b, c):        # Aligning the cell vectors of the orthogonal cell along x,y and z axes.

    with open(poscar_file, 'r') as file:
        poscar_lines = file.readlines()
    
    poscar_lines[2] = f"  {a}   0.0000000000000000   0.0000000000000000\n"
    poscar_lines[3] = f"  0.0000000000000000   {b}   0.0000000000000000\n"
    poscar_lines[4] = f"  0.0000000000000000   0.0000000000000000   {c}\n"
    
    with open(poscar_file, 'w') as file:
        file.writelines( poscar_lines )

In [6]:
parent_directory = os.getcwd()

In [11]:

print(parent_directory)
Order2 = ['Pristine']+[x+1 for x in range(7)]

Interfaces = ['100','110','111','211','210','221']

for interface in Interfaces:
    print("===============================================================================================")
    print(interface)
    print("===============================================================================================")
    print("\n\n")
    sub_dir = os.path.join(parent_directory, interface)  
    os.chdir(sub_dir)

    Occupied, indices, s = Resources()
    print(Occupied)
    s = Structure_Editor(s, indices)
    
    Lattice = s.lattice
    fraction = 0.960920049102452           #resize the vectors to come closer to the delithiated lattice parameter.
    a, b, c = fraction*Lattice.a , fraction*Lattice.b , fraction*Lattice.c
    
    O_2 = indices['O2'].copy()
    o1 = 0
    for o2 in Order2:
        
        if o2=='Pristine':
            orient2=0
        else:
            orient2=o2

        Mn_l, Vacancy_l = Interface_Cell_Occupancies(o1, orient2, Occupied, 0.5, s)   
        Vac_l = Vacancy_l+indices['tet'].copy()
        Li_l = []                                 # for the fully delithiated structure
        
        chgnet_structure = s.copy()

        poscar_file = f'POSCAR_{interface}_Interface_with_{o2}'
        
        # Not checking the stoichiometry of the cell before writing POSCAR. You might need it. 

        Structure_Maker(Li_l, Vac_l, Mn_l, poscar_file, chgnet_structure)    
        POSCAR_edit(poscar_file, a, b, c)

/Users/tarawork/worklocal/postdoc_lbl/Theory/theory_character/Delta_theory/Generate_structures/run_again
100



3.429286 -4.849742 5.939697
6.858571 4.849742 0.000000
-10.287857 14.549227 17.819091


AttributeError: 'Composition' object has no attribute '_n_atoms'

In [ ]:
parent_directory